In [2]:
from cryoS2Sdrop.analyze import *
from tqdm import tqdm

PARENT_PATH = setup.PARENT_PATH

pd.options.display.float_format = "{:,.6f}".format
pd.set_option('display.max_columns', 50)

%matplotlib inline
%config Completer.use_jedi = False
%load_ext autoreload
%autoreload 2

In [3]:
tomo_list = [
    'shrec2021_model4_dummy'
]
exp_list = [
    "realBernoulli_convolved_comparison",
    "realBernoulli_volMask_comparison",
    "fourierBernoulli_dropoutLevel_comparison",
    "fourierHighFreqMask_comparison"
]

data_log = []

for exp_name in exp_list:
    print(exp_name)
    for tomo in tqdm(tomo_list):

        logdir = 'data/S2SDenoising/model_logs/%s/%s/' %(tomo, exp_name)
        logdir = os.path.join(PARENT_PATH, logdir)

        _data_log = logdir_to_dataframe(logdir, clip_values=True)
        data_log.append(_data_log)

data_log = pd.concat(data_log)
data_log = data_log.reset_index().drop('index', 1)
data_log.head()

realBernoulli_convolved_comparison


100%|████████████████████████████████████████████████████████████| 1/1 [00:14<00:00, 14.50s/it]


realBernoulli_volMask_comparison


100%|████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.66s/it]


fourierBernoulli_dropoutLevel_comparison


100%|████████████████████████████████████████████████████████████| 1/1 [00:13<00:00, 13.37s/it]


fourierHighFreqMask_comparison


100%|████████████████████████████████████████████████████████████| 1/1 [00:10<00:00, 10.04s/it]


,model,version,dataset,TV_alpha,Version_comment,transform,full_tomo_psnr,full_tomo_ssim,baseline_psnr,baseline_ssim,tomo_path,gt_tomo_path,use_deconv_as_target,predict_simRecon,use_deconv_data,p,n2v_psnr,n2v_ssim,baseline_psnr_best,baseline_ssim_best,ssim_vs_baseline,n2v_ssim_vs_baseline,psnr_vs_baseline,n2v_psnr_vs_baseline,noise_level,tomo_name,pred_path
0,realBernoulli_convolved_comparison,version_0,singleCET_dataset,0,Bernoulli,{'p': '0.5'},6.016479,-0.039001,5.503306,-0.038987,/home/ubuntu/Thesis/data/S2SDenoising/dummy_to...,/home/ubuntu/Thesis/data/S2SDenoising/dummy_to...,null,null,false,0.3,5.716530,-0.037105,5.503306,-0.038987,0.034781,-4.828497,9.324806,3.874470,other,shrec2021_model4_dummy,/home/ubuntu/Thesis/data/S2SDenoising/model_lo...
1,realBernoulli_convolved_comparison,version_1,singleCET_dataset,0,Deconvolved Bernoulli,{'p': '0.5'},5.192185,-0.056545,5.213845,-0.053055,/home/ubuntu/Thesis/data/S2SDenoising/dummy_to...,/home/ubuntu/Thesis/data/S2SDenoising/dummy_to...,null,null,true,0.3,5.716530,-0.037105,5.503306,-0.038987,45.033288,-4.828497,-5.653347,3.874470,other,shrec2021_model4_dummy,/home/ubuntu/Thesis/data/S2SDenoising/model_lo...
2,realBernoulli_convolved_comparison,version_2,singleCET_dataset,0,Bernoulli,{'p': '0.5'},5.751411,-0.047679,5.503306,-0.038987,/home/ubuntu/Thesis/data/S2SDenoising/dummy_to...,/home/ubuntu/Thesis/data/S2SDenoising/dummy_to...,null,null,false,0.7,5.716530,-0.037105,5.503306,-0.038987,22.293346,-4.828497,4.508282,3.874470,other,shrec2021_model4_dummy,/home/ubuntu/Thesis/data/S2SDenoising/model_lo...
3,realBernoulli_convolved_comparison,version_3,singleCET_dataset,0,Deconvolved Bernoulli,{'p': '0.5'},4.963487,-0.059713,5.213845,-0.053055,/home/ubuntu/Thesis/data/S2SDenoising/dummy_to...,/home/ubuntu/Thesis/data/S2SDenoising/dummy_to...,null,null,true,0.7,5.716530,-0.037105,5.503306,-0.038987,53.159546,-4.828497,-9.808998,3.874470,other,shrec2021_model4_dummy,/home/ubuntu/Thesis/data/S2SDenoising/model_lo...
4,realBernoulli_volMask_comparison,version_0,singleCET_dataset,0,Volumetric mask(0.3) with prob=1,{'p': '0.5'},6.162219,-0.040156,5.503306,-0.038987,/home/ubuntu/Thesis/data/S2SDenoising/dummy_to...,/home/ubuntu/Thesis/data/S2SDenoising/dummy_to...,null,null,false,0.3,5.716530,-0.037105,5.503306,-0.038987,2.997406,-4.828497,11.973032,3.874470,other,shrec2021_model4_dummy,/home/ubuntu/Thesis/data/S2SDenoising/model_lo...


In [8]:
df = data_log.groupby(['model', 'noise_level', 'p', 'Version_comment'])['full_tomo_ssim', 'full_tomo_psnr'].max().reset_index()

df2 = data_log.groupby(['model', 'p', 'Version_comment', 'noise_level'])['n2v_ssim', 'n2v_psnr'].mean().reset_index()
df2['model'] = "N2V"
df2 = df2[['model', 'noise_level', 'p', 'Version_comment', 'n2v_ssim', 'n2v_psnr']]
df2.columns = ['model', 'noise_level', 'p', 'Version_comment', 'full_tomo_ssim', 'full_tomo_psnr']

df = pd.concat([df, df2]).sort_values(['noise_level', 'p', 'model', 'Version_comment'])

df.set_index(['noise_level', 'p']).head()

model              Version_comment  full_tomo_ssim  \
noise_level p                                                        
other       0.3   N2V                    Bernoulli       -0.037105   
            0.3   N2V        Deconvolved Bernoulli       -0.037105   
            0.3   N2V            Fourier Bernoulli       -0.037105   
            0.3   N2V  Fourier with hiFreqMask=0.5       -0.037105   
            0.3   N2V  Fourier with hiFreqMask=0.7       -0.037105   

                 full_tomo_psnr  
noise_level p                    
other       0.3        5.716530  
            0.3        5.716530  
            0.3        5.716530  
            0.3        5.716530  
            0.3        5.716530

In [10]:
for n, g in df.groupby(['noise_level']):
    aux_df = g.sort_values(['full_tomo_ssim'], ascending=False).drop_duplicates(['full_tomo_ssim'])
    display(aux_df)

,model,noise_level,p,Version_comment,full_tomo_ssim,full_tomo_psnr
5,N2V,other,0.3,Bernoulli,-0.037105,5.716530
5,realBernoulli_convolved_comparison,other,0.3,Bernoulli,-0.039001,6.016479
10,realBernoulli_volMask_comparison,other,0.3,Volumetric mask(0.3) with prob=1,-0.040156,6.162219
9,realBernoulli_volMask_comparison,other,0.3,Volumetric mask(0.3) with prob=0.5,-0.040177,6.126647
0,fourierBernoulli_dropoutLevel_comparison,other,0.3,Fourier Bernoulli,-0.041031,5.881138
7,realBernoulli_convolved_comparison,other,0.7,Bernoulli,-0.047679,5.751411
2,fourierHighFreqMask_comparison,other,0.3,Fourier with hiFreqMask=0.5,-0.050725,5.890501
1,fourierBernoulli_dropoutLevel_comparison,other,0.7,Fourier Bernoulli,-0.051183,5.254138
3,fourierHighFreqMask_comparison,other,0.3,Fourier with hiFreqMask=0.7,-0.052216,5.385791
4,fourierHighFreqMask_comparison,other,0.3,Fourier with hiFreqMask=1,-0.056045,5.399857
